In [1]:
using Pkg
Pkg.activate(".")

using JuMP
using DataFrames
using CSV
using Gurobi
using DataFrames
using Random

include("./parameters.jl")
include("./get_functions.jl")
include("./modification_functions.jl")

construct_ef_model (generic function with 1 method)

In [9]:
Random.seed!(6)
busorder = shuffle(101:124)

adjustment = Float64[];
for i = 1:24
    push!(adjustment, (-0.0115+(i-1)*0.001)/100)
    println(adjustment[i])
end

busadjdict = Dict()

for i = 1:24
    println(busorder[i], " ", adjustment[i])
    busadjdict[busorder[i]] = adjustment[i]
end


-0.000115
-0.00010499999999999999
-9.499999999999999e-5
-8.5e-5
-7.5e-5
-6.5e-5
-5.4999999999999995e-5
-4.4999999999999996e-5
-3.5e-5
-2.4999999999999988e-5
-1.4999999999999995e-5
-5.000000000000005e-6
5.000000000000005e-6
1.5000000000000014e-5
2.5000000000000005e-5
3.5e-5
4.5e-5
5.5000000000000016e-5
6.500000000000002e-5
7.5e-5
8.5e-5
9.500000000000002e-5
0.00010499999999999999
0.000115
121 -0.000115
114 -0.00010499999999999999
107 -9.499999999999999e-5
104 -8.5e-5
116 -7.5e-5
110 -6.5e-5
124 -5.4999999999999995e-5
119 -4.4999999999999996e-5
120 -3.5e-5
109 -2.4999999999999988e-5
118 -1.4999999999999995e-5
113 -5.000000000000005e-6
111 5.000000000000005e-6
106 1.5000000000000014e-5
117 2.5000000000000005e-5
115 3.5e-5
108 4.5e-5
112 5.5000000000000016e-5
105 6.500000000000002e-5
123 7.5e-5
102 8.5e-5
103 9.500000000000002e-5
101 0.00010499999999999999
122 0.000115


In [278]:
efmodel = read_from_file("ef_12scen_test_revised_noint_PR.mps")
JuMP.set_optimizer(efmodel, Gurobi.Optimizer)
JuMP.set_optimizer_attribute(efmodel, "Method", 1) 

Academic license - for non-commercial use only - expires 2022-08-19


1

In [10]:
model2 = [];
for i = 1:1#2
    push!(model2, JuMP.read_from_file("./storage_expansion_revised/second_stage/noint_PR_exp3_scen_$(i).mps") )
end

In [256]:
budget = get_expansion_budget(efmodel, 0)

set_normalized_rhs(budget, 3500)

budget

expansion_budget_0 : 57.62 PR_{101}_0 + 57.62 PR_{102}_0 + 57.62 PR_{103}_0 + 57.62 PR_{104}_0 + 57.62 PR_{105}_0 + 57.62 PR_{106}_0 + 57.62 PR_{107}_0 + 57.62 PR_{108}_0 + 57.62 PR_{109}_0 + 57.62 PR_{110}_0 + 57.62 PR_{111}_0 + 57.62 PR_{112}_0 + 57.62 PR_{113}_0 + 57.62 PR_{114}_0 + 57.62 PR_{115}_0 + 57.62 PR_{116}_0 + 57.62 PR_{117}_0 + 57.62 PR_{118}_0 + 57.62 PR_{119}_0 + 57.62 PR_{120}_0 + 57.62 PR_{121}_0 + 57.62 PR_{122}_0 + 57.62 PR_{123}_0 + 57.62 PR_{124}_0 <= 3500.0

In [257]:
#=for sc = 1:12
    for bus in buses
        for ts in timesteps
            varch = get_charging_variable(efmodel, bus, ts, sc)
            vardi = get_discharging_variable(efmodel, bus, ts, sc)
            
            con = get_storage_balance(efmodel, bus, ts, sc)
            
            coeffch = normalized_coefficient(con, varch)
            coeffdi = normalized_coefficient(con, vardi)
            
            set_normalized_coefficient(con, varch, (1+busadjdict[bus])*coeffch)
            set_normalized_coefficient(con, vardi, coeffdi/(1+busadjdict[bus]))
            
        end
    end
end=#

In [258]:
get_charge_ub(efmodel, 101, 1,1)

Cub_{101,1}_1 : -PR_{101}_0 + Pch_{101_STORAGE_1,1}_1 <= 0.0

In [259]:
get_discharge_ub(efmodel, 101, 1, 1)

Dub_{101,1}_1 : -PR_{101}_0 + Pdis_{101_STORAGE_1,1}_1 <= 0.0

In [260]:
get_storage_ub(efmodel, 101, 1, 1)

Eub_{101,1}_1 : Est_{101_STORAGE_1,1}_1 - 5 PR_{101}_0 <= 0.0

In [261]:
for sc = 1:12
    for bus in buses
        var = get_PR_variable(efmodel, bus, 0)
        for ts in timesteps
            conc = get_charge_ub(efmodel, bus, ts, sc)
            cond = get_discharge_ub(efmodel, bus, ts, sc)
            cone = get_storage_ub(efmodel, bus, ts, sc)
            
            valc = JuMP.normalized_coefficient(conc, var)
            vald = JuMP.normalized_coefficient(cond, var)
            vale = JuMP.normalized_coefficient(cone, var)
            
            JuMP.set_normalized_coefficient(conc, var, (1+busadjdict[bus])*valc)
            JuMP.set_normalized_coefficient(cond, var, (1+busadjdict[bus])*vald)
            JuMP.set_normalized_coefficient(cone, var, (1+busadjdict[bus])*vale)
        end
    end
end

In [262]:
optimize!(efmodel)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 129625 rows, 52728 columns and 1439760 nonzeros
Model fingerprint: 0x505f8405
Coefficient statistics:
  Matrix range     [2e-04, 6e+01]
  Objective range  [5e-03, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-03, 4e+03]
Presolve removed 86772 rows and 3800 columns
Presolve time: 0.82s
Presolved: 42853 rows, 49756 columns, 1006968 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.446348e+03   0.000000e+00      1s
    6696    3.6157270e+02   3.387327e+05   0.000000e+00      5s
    9511    4.7084018e+02   6.631150e+05   0.000000e+00     10s
   11760    4.8448763e+02   1.533491e+05   0.000000e+00     15s
   13890    4.9069401e+02   2.958969e+05   0.000000e+00     20s
   16786    5.0072082e+02

In [263]:
sum = 0.
for bus in buses
    var = get_PR_variable(efmodel, bus, 0)
    println(var, " ",value(var))
    sum+=value(var)
end
println(sum)
println()
println(objective_value(efmodel))

PR_{101}_0 3.4092269572133995
PR_{102}_0 0.05971038351689665
PR_{103}_0 0.3600696306461533
PR_{104}_0 0.0
PR_{105}_0 0.0
PR_{106}_0 0.0
PR_{107}_0 0.0
PR_{108}_0 0.0
PR_{109}_0 0.0
PR_{110}_0 0.0
PR_{111}_0 0.0
PR_{112}_0 0.0
PR_{113}_0 0.0
PR_{114}_0 0.9670400701147679
PR_{115}_0 0.0
PR_{116}_0 1.7968804667448384
PR_{117}_0 1.7039088057263998
PR_{118}_0 0.0
PR_{119}_0 0.0
PR_{120}_0 0.0
PR_{121}_0 0.0
PR_{122}_0 0.5589118329165429
PR_{123}_0 0.0
PR_{124}_0 0.0
8.855748146878998

768.1506940035111


In [201]:
#=for sc = 1:12
    for bus in buses
        for ts in timesteps
            varch = get_charging_variable(model2[sc], bus, ts)
            vardi = get_discharging_variable(model2[sc], bus, ts)
            
            con = get_storage_balance(model2[sc], bus, ts)
            
            coeffch = normalized_coefficient(con, varch)
            coeffdi = normalized_coefficient(con, vardi)
            
            set_normalized_coefficient(con, varch, (1+busadjdict[bus])*coeffch)
            set_normalized_coefficient(con, vardi, coeffdi/(1+busadjdict[bus]))
            
        end
    end
end=#

In [11]:
for sc = 1:1#2
    for bus in buses
        var = get_PR_variable(model2[sc], bus)
        for ts in timesteps
            conc = get_charge_ub(model2[sc], bus, ts)
            cond = get_discharge_ub(model2[sc], bus, ts)
            cone = get_storage_ub(model2[sc], bus, ts)
            
            valc = JuMP.normalized_coefficient(conc, var)
            vald = JuMP.normalized_coefficient(cond, var)
            vale = JuMP.normalized_coefficient(cone, var)
            
            JuMP.set_normalized_coefficient(conc, var, (1+busadjdict[bus])*valc)
            JuMP.set_normalized_coefficient(cond, var, (1+busadjdict[bus])*vald)
            JuMP.set_normalized_coefficient(cone, var, (1+busadjdict[bus])*vale)
        end
    end
end

In [12]:
#get_storage_balance(model2[7], 101, 1)

In [13]:
println(get_charge_ub(model2[1], 101, 1))
println(get_discharge_ub(model2[1], 101, 1))
println(get_storage_ub(model2[1], 101, 1))

Cub_{101,1} : Pch_{101_STORAGE_1,1} - 1.000105 PR_{101} <= 0.0
Dub_{101,1} : Pdis_{101_STORAGE_1,1} - 1.000105 PR_{101} <= 0.0
Eub_{101,1} : Est_{101_STORAGE_1,1} - 5.000525 PR_{101} <= 0.0


In [14]:
for i = 1:1#2
    JuMP.write_to_file(model2[i], "./storage_expansion_revised/second_stage/noint_PR_exp36p00015_scen_$(i).mps") 
end

for bus in buses
    println(bus, " ", busadjdict[bus])
end

In [286]:
for (key, value) in objective_function(model2[7]).terms
    println(key, " ", value)
end

Pth_{123_STEAM_2,1} 0.9005022658968751
Pth_{101_STEAM_3,1} 0.7057312506149999
Pth_{123_CT_1,1} 1.247549007985875
Pth_{113_CT_3,1} 1.257345677010375
Pth_{102_STEAM_3,1} 0.7882917691839374
Pth_{113_CT_2,1} 1.257345677010375
Pth_{101_STEAM_4,1} 0.7057312506149999
Pth_{113_CT_1,1} 1.257345677010375
Pth_{113_CT_4,1} 1.257345677010375
Pth_{118_CC_1,1} 1.0136002922595
Pth_{115_STEAM_3,1} 0.8596177257217498
Pth_{101_CT_1,1} 4.0067364450825
Pth_{115_STEAM_2,1} 5.1377478554025
Pth_{101_CT_2,1} 4.0067364450825
Pth_{123_STEAM_3,1} 0.8278443767471249
Pth_{123_CT_5,1} 1.247549007985875
Pth_{116_STEAM_1,1} 0.8493869389923749
Pth_{115_STEAM_1,1} 5.1377478554025
Pth_{121_NUCLEAR_1,1} 0.2959803375
Pth_{102_CT_2,1} 3.96959678229375
Pth_{123_CT_4,1} 1.247549007985875
Pth_{102_CT_1,1} 3.96959678229375
Pth_{102_STEAM_4,1} 0.7882917691839374
Pth_{107_CC_1,1} 0.9916642724872501
Pth_{123_STEAM_2,2} 0.9005022658968751
Pth_{101_STEAM_3,2} 0.7057312506149999
Pth_{123_CT_1,2} 1.247549007985875
Pth_{113_CT_3,2} 1.2

Pth_{121_NUCLEAR_1,10} 0.2959803375
Pth_{102_CT_2,10} 3.96959678229375
Pth_{123_CT_4,10} 1.247549007985875
Pth_{102_CT_1,10} 3.96959678229375
Pth_{102_STEAM_4,10} 0.7882917691839374
Pth_{107_CC_1,10} 0.9916642724872501
Pth_{123_STEAM_2,11} 0.9005022658968751
Pth_{101_STEAM_3,11} 0.7057312506149999
Pth_{123_CT_1,11} 1.247549007985875
Pth_{113_CT_3,11} 1.257345677010375
Pth_{102_STEAM_3,11} 0.7882917691839374
Pth_{113_CT_2,11} 1.257345677010375
Pth_{101_STEAM_4,11} 0.7057312506149999
Pth_{113_CT_1,11} 1.257345677010375
Pth_{113_CT_4,11} 1.257345677010375
Pth_{118_CC_1,11} 1.0136002922595
Pth_{115_STEAM_3,11} 0.8596177257217498
Pth_{101_CT_1,11} 4.0067364450825
Pth_{115_STEAM_2,11} 5.1377478554025
Pth_{101_CT_2,11} 4.0067364450825
Pth_{123_STEAM_3,11} 0.8278443767471249
Pth_{123_CT_5,11} 1.247549007985875
Pth_{116_STEAM_1,11} 0.8493869389923749
Pth_{115_STEAM_1,11} 5.1377478554025
Pth_{121_NUCLEAR_1,11} 0.2959803375
Pth_{102_CT_2,11} 3.96959678229375
Pth_{123_CT_4,11} 1.247549007985875
Pt

Pth_{102_STEAM_3,20} 0.7882917691839374
Pth_{113_CT_2,20} 1.257345677010375
Pth_{101_STEAM_4,20} 0.7057312506149999
Pth_{113_CT_1,20} 1.257345677010375
Pth_{113_CT_4,20} 1.257345677010375
Pth_{118_CC_1,20} 1.0136002922595
Pth_{115_STEAM_3,20} 0.8596177257217498
Pth_{101_CT_1,20} 4.0067364450825
Pth_{115_STEAM_2,20} 5.1377478554025
Pth_{101_CT_2,20} 4.0067364450825
Pth_{123_STEAM_3,20} 0.8278443767471249
Pth_{123_CT_5,20} 1.247549007985875
Pth_{116_STEAM_1,20} 0.8493869389923749
Pth_{115_STEAM_1,20} 5.1377478554025
Pth_{121_NUCLEAR_1,20} 0.2959803375
Pth_{102_CT_2,20} 3.96959678229375
Pth_{123_CT_4,20} 1.247549007985875
Pth_{102_CT_1,20} 3.96959678229375
Pth_{102_STEAM_4,20} 0.7882917691839374
Pth_{107_CC_1,20} 0.9916642724872501
Pth_{123_STEAM_2,21} 0.9005022658968751
Pth_{101_STEAM_3,21} 0.7057312506149999
Pth_{123_CT_1,21} 1.247549007985875
Pth_{113_CT_3,21} 1.257345677010375
Pth_{102_STEAM_3,21} 0.7882917691839374
Pth_{113_CT_2,21} 1.257345677010375
Pth_{101_STEAM_4,21} 0.7057312506

Pdis_{107_STORAGE_1,5} 0.054787499999999996
Pdis_{108_STORAGE_1,5} 0.054787499999999996
Pdis_{109_STORAGE_1,5} 0.054787499999999996
Pdis_{110_STORAGE_1,5} 0.054787499999999996
Pdis_{111_STORAGE_1,5} 0.054787499999999996
Pdis_{112_STORAGE_1,5} 0.054787499999999996
Pdis_{113_STORAGE_1,5} 0.054787499999999996
Pdis_{114_STORAGE_1,5} 0.054787499999999996
Pdis_{115_STORAGE_1,5} 0.054787499999999996
Pdis_{116_STORAGE_1,5} 0.054787499999999996
Pdis_{117_STORAGE_1,5} 0.054787499999999996
Pdis_{118_STORAGE_1,5} 0.054787499999999996
Pdis_{119_STORAGE_1,5} 0.054787499999999996
Pdis_{120_STORAGE_1,5} 0.054787499999999996
Pdis_{121_STORAGE_1,5} 0.054787499999999996
Pdis_{122_STORAGE_1,5} 0.054787499999999996
Pdis_{123_STORAGE_1,5} 0.054787499999999996
Pdis_{124_STORAGE_1,5} 0.054787499999999996
Pdis_{101_STORAGE_1,6} 0.054787499999999996
Pdis_{102_STORAGE_1,6} 0.054787499999999996
Pdis_{103_STORAGE_1,6} 0.054787499999999996
Pdis_{104_STORAGE_1,6} 0.054787499999999996
Pdis_{105_STORAGE_1,6} 0.0547874

Pdis_{113_STORAGE_1,13} 0.054787499999999996
Pdis_{114_STORAGE_1,13} 0.054787499999999996
Pdis_{115_STORAGE_1,13} 0.054787499999999996
Pdis_{116_STORAGE_1,13} 0.054787499999999996
Pdis_{117_STORAGE_1,13} 0.054787499999999996
Pdis_{118_STORAGE_1,13} 0.054787499999999996
Pdis_{119_STORAGE_1,13} 0.054787499999999996
Pdis_{120_STORAGE_1,13} 0.054787499999999996
Pdis_{121_STORAGE_1,13} 0.054787499999999996
Pdis_{122_STORAGE_1,13} 0.054787499999999996
Pdis_{123_STORAGE_1,13} 0.054787499999999996
Pdis_{124_STORAGE_1,13} 0.054787499999999996
Pdis_{101_STORAGE_1,14} 0.054787499999999996
Pdis_{102_STORAGE_1,14} 0.054787499999999996
Pdis_{103_STORAGE_1,14} 0.054787499999999996
Pdis_{104_STORAGE_1,14} 0.054787499999999996
Pdis_{105_STORAGE_1,14} 0.054787499999999996
Pdis_{106_STORAGE_1,14} 0.054787499999999996
Pdis_{107_STORAGE_1,14} 0.054787499999999996
Pdis_{108_STORAGE_1,14} 0.054787499999999996
Pdis_{109_STORAGE_1,14} 0.054787499999999996
Pdis_{110_STORAGE_1,14} 0.054787499999999996
Pdis_{111_

Pdis_{103_STORAGE_1,22} 0.054787499999999996
Pdis_{104_STORAGE_1,22} 0.054787499999999996
Pdis_{105_STORAGE_1,22} 0.054787499999999996
Pdis_{106_STORAGE_1,22} 0.054787499999999996
Pdis_{107_STORAGE_1,22} 0.054787499999999996
Pdis_{108_STORAGE_1,22} 0.054787499999999996
Pdis_{109_STORAGE_1,22} 0.054787499999999996
Pdis_{110_STORAGE_1,22} 0.054787499999999996
Pdis_{111_STORAGE_1,22} 0.054787499999999996
Pdis_{112_STORAGE_1,22} 0.054787499999999996
Pdis_{113_STORAGE_1,22} 0.054787499999999996
Pdis_{114_STORAGE_1,22} 0.054787499999999996
Pdis_{115_STORAGE_1,22} 0.054787499999999996
Pdis_{116_STORAGE_1,22} 0.054787499999999996
Pdis_{117_STORAGE_1,22} 0.054787499999999996
Pdis_{118_STORAGE_1,22} 0.054787499999999996
Pdis_{119_STORAGE_1,22} 0.054787499999999996
Pdis_{120_STORAGE_1,22} 0.054787499999999996
Pdis_{121_STORAGE_1,22} 0.054787499999999996
Pdis_{122_STORAGE_1,22} 0.054787499999999996
Pdis_{123_STORAGE_1,22} 0.054787499999999996
Pdis_{124_STORAGE_1,22} 0.054787499999999996
Pdis_{101_

OL_{110,7} 73.05
LOL_{111,7} 310.4625
OL_{111,7} 73.05
LOL_{112,7} 310.4625
OL_{112,7} 73.05
LOL_{113,7} 310.4625
OL_{113,7} 73.05
LOL_{114,7} 310.4625
OL_{114,7} 73.05
LOL_{115,7} 310.4625
OL_{115,7} 73.05
LOL_{116,7} 310.4625
OL_{116,7} 73.05
LOL_{117,7} 310.4625
OL_{117,7} 73.05
LOL_{118,7} 310.4625
OL_{118,7} 73.05
LOL_{119,7} 310.4625
OL_{119,7} 73.05
LOL_{120,7} 310.4625
OL_{120,7} 73.05
LOL_{121,7} 310.4625
OL_{121,7} 73.05
LOL_{122,7} 310.4625
OL_{122,7} 73.05
LOL_{123,7} 310.4625
OL_{123,7} 73.05
LOL_{124,7} 310.4625
OL_{124,7} 73.05
LOL_{101,8} 310.4625
OL_{101,8} 73.05
LOL_{102,8} 310.4625
OL_{102,8} 73.05
LOL_{103,8} 310.4625
OL_{103,8} 73.05
LOL_{104,8} 310.4625
OL_{104,8} 73.05
LOL_{105,8} 310.4625
OL_{105,8} 73.05
LOL_{106,8} 310.4625
OL_{106,8} 73.05
LOL_{107,8} 310.4625
OL_{107,8} 73.05
LOL_{108,8} 310.4625
OL_{108,8} 73.05
LOL_{109,8} 310.4625
OL_{109,8} 73.05
LOL_{110,8} 310.4625
OL_{110,8} 73.05
LOL_{111,8} 310.4625
OL_{111,8} 73.05
LOL_{112,8} 310.4625
OL_{112,8} 7

LOL_{111,16} 310.4625
OL_{111,16} 73.05
LOL_{112,16} 310.4625
OL_{112,16} 73.05
LOL_{113,16} 310.4625
OL_{113,16} 73.05
LOL_{114,16} 310.4625
OL_{114,16} 73.05
LOL_{115,16} 310.4625
OL_{115,16} 73.05
LOL_{116,16} 310.4625
OL_{116,16} 73.05
LOL_{117,16} 310.4625
OL_{117,16} 73.05
LOL_{118,16} 310.4625
OL_{118,16} 73.05
LOL_{119,16} 310.4625
OL_{119,16} 73.05
LOL_{120,16} 310.4625
OL_{120,16} 73.05
LOL_{121,16} 310.4625
OL_{121,16} 73.05
LOL_{122,16} 310.4625
OL_{122,16} 73.05
LOL_{123,16} 310.4625
OL_{123,16} 73.05
LOL_{124,16} 310.4625
OL_{124,16} 73.05
LOL_{101,17} 310.4625
OL_{101,17} 73.05
LOL_{102,17} 310.4625
OL_{102,17} 73.05
LOL_{103,17} 310.4625
OL_{103,17} 73.05
LOL_{104,17} 310.4625
OL_{104,17} 73.05
LOL_{105,17} 310.4625
OL_{105,17} 73.05
LOL_{106,17} 310.4625
OL_{106,17} 73.05
LOL_{107,17} 310.4625
OL_{107,17} 73.05
LOL_{108,17} 310.4625
OL_{108,17} 73.05
LOL_{109,17} 310.4625
OL_{109,17} 73.05
LOL_{110,17} 310.4625
OL_{110,17} 73.05
LOL_{111,17} 310.4625
OL_{111,17} 73.05


PR_{109} 57.62
PR_{110} 57.62
PR_{111} 57.62
PR_{112} 57.62
PR_{113} 57.62
PR_{114} 57.62
PR_{115} 57.62
PR_{116} 57.62
PR_{117} 57.62
PR_{118} 57.62
PR_{119} 57.62
PR_{120} 57.62
PR_{121} 57.62
PR_{122} 57.62
PR_{123} 57.62
PR_{124} 57.62
